# SQL CARTO QUERY FUNCTION
# Written for PUI2016_Python2

#### SQL homework for PUI Homework 7
#### Submission by #JKtours (jlk635)


In [1]:
SQL_SOURCE = 'https://jlk635.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-513527a0484b>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

## Task 1 — Familiarize with SQL Clauses

#### Sort data by start_station_id, tripduration 
       - Only checking trips with duration <= 3 hours 

In [ ]:
qry2 = '''
SELECT *
FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id, tripduration 
'''

In [ ]:
dfTripDur = pd.read_csv(StringIO.StringIO(queryCartoDB(qry2)), sep=',')

In [ ]:
dfTripDur.head()

#### Only show the top/last 10 records (aka head and tail in SQL) 

Head

In [ ]:
qry3 = '''
SELECT  *
FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id, tripduration 
LIMIT 10
'''


In [ ]:
dfTripDurHead = pd.read_csv(StringIO.StringIO(queryCartoDB(qry3)), sep=',')

In [ ]:
dfTripDurHead

Tail

In [ ]:
qry4 = '''
SELECT  *
FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id DESC, tripduration DESC
LIMIT 10
'''

In [ ]:
dfTripDurTail = pd.read_csv(StringIO.StringIO(queryCartoDB(qry4)), sep=',')
dfTripDurTail

#### List all unique start_station_id values

In [ ]:
qry5 = '''
SELECT DISTINCT start_station_id
FROM citibike
ORDER BY start_station_id 
'''

In [ ]:
dfSSID = pd.read_csv(StringIO.StringIO(queryCartoDB(qry5)), sep=',')
dfSSID.drop(['Unnamed: 1'], axis = 1, inplace=True)

In [ ]:
dfSSID.head(15)

#### Aggregation functions: 
    - Count the number of trips (aka wc -l in SQL) 
    - Find the average/min/max trip duration

In [ ]:
qry6 = '''
SELECT COUNT(starttime) as Counts, AVG(tripduration) as AvgDuration, MAX(tripduration) as MaxDuration, MIN(tripduration) as MinDuration
FROM citibike
WHERE tripduration <10800
'''

In [ ]:
dfTripStats = pd.read_csv(StringIO.StringIO(queryCartoDB(qry6)), sep=',')

dfTripStats

# Task 2 — Working with date/time
   
#### Selecting trips started on Feb-02-2015 only 
    

In [ ]:
qry7 = '''
SELECT *
FROM citibike
WHERE starttime > '2015-02-02' AND starttime < '2015-02-03'
'''

In [ ]:
dfFeb2 = pd.read_csv(StringIO.StringIO(queryCartoDB(qry7)), sep=',')

dfFeb2.head()

In [ ]:
dfFeb2.shape

#### Selecting trips started on the weekends 
What are average trip duration during weekends? 


In [ ]:


qry8 = '''
SELECT EXTRACT(ISODOW FROM  citibike.starttime), citibike.tripduration 
FROM citibike
WHERE EXTRACT(ISODOW FROM  citibike.starttime) >= 6
'''


In [ ]:
dfWeekendTrips = pd.read_csv(StringIO.StringIO(queryCartoDB(qry8)), sep=',')

dfWeekendTrips.head()

In [ ]:
qry9 = '''
SELECT AVG(tripduration) 
FROM citibike
WHERE EXTRACT(ISODOW FROM  citibike.starttime) >= 6
'''

In [ ]:
dfAvgDurationWeekendTrips = pd.read_csv(StringIO.StringIO(queryCartoDB(qry9)), sep=',')
dfAvgDurationWeekendTrips.drop(['Unnamed: 1'], axis = 1, inplace=True)
dfAvgDurationWeekendTrips


In [ ]:
## Another query approach provides slightly different answer

qry20 = '''
SELECT AVG(tripduration) 
FROM citibike
WHERE EXTRACT(DOW FROM  citibike.starttime) >= 5
'''

In [ ]:
dets = pd.read_csv(StringIO.StringIO(queryCartoDB(qry20)), sep=',')
dets.drop(['Unnamed: 1'], axis = 1, inplace=True)
dets

#### Can we do the same for weekday?

In [ ]:
qry10 = '''
SELECT AVG(tripduration) 
FROM citibike
WHERE EXTRACT(ISODOW FROM  citibike.starttime) < 6
'''

In [ ]:
dfAvgDurationWeekdayTrips = pd.read_csv(StringIO.StringIO(queryCartoDB(qry10)), sep=',')
dfAvgDurationWeekdayTrips.drop(['Unnamed: 1'], axis = 1, inplace=True)
dfAvgDurationWeekdayTrips


# Task 3 — Working with Space
#### Showing the list of start station locations 
    Using GROUP BY 


In [ ]:

qry11 = '''
SELECT DISTINCT start_station_id, start_station_latitude, start_station_longitude
FROM citibike
GROUP BY start_station_id, start_station_latitude, start_station_longitude
'''

In [ ]:
dfSSLocs = pd.read_csv(StringIO.StringIO(queryCartoDB(qry11)), sep=',')
dfSSLocs.head()

In [ ]:
dfSSLocs.shape

#### Showing the number of trips started per station 
    ... but only for stations within 500m of Time Square! 

    The coordinates of Times Square is (40.7577,-73.9857)

In [ ]:

qry12 = '''
SELECT DISTINCT start_station_id, start_station_latitude, start_station_longitude, COUNT(starttime) AS tripcount
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                       start_station_longitude 
                                             )   :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
GROUP BY start_station_id, start_station_latitude, start_station_longitude
'''



In [ ]:
dfSSLocsNrTimesSqu = pd.read_csv(StringIO.StringIO(queryCartoDB(qry12)), sep=',')
dfSSLocsNrTimesSqu

# Task 4 — Putting it all together
#### Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare! 


In [ ]:
qry13 = '''
SELECT DISTINCT start_station_id, start_station_latitude, start_station_longitude, COUNT(starttime) AS tripcount, AVG(tripduration) AS avgtripdur
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                       start_station_longitude 
                                            )   :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
                 
                 AND
                 
                 EXTRACT(DOW FROM  citibike.starttime) >= 5
                 
GROUP BY start_station_id, start_station_latitude, start_station_longitude
'''

In [ ]:
dfAvgTripDur_WE_TS = pd.read_csv(StringIO.StringIO(queryCartoDB(qry13)), sep=',')
dfAvgTripDur_WE_TS

In [ ]:
qry14 = '''
SELECT MAX(avgtripdur)

FROM (

SELECT DISTINCT start_station_id, start_station_latitude, start_station_longitude, COUNT(starttime) AS tripcount, AVG(tripduration) AS avgtripdur
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                       start_station_longitude 
                                                )   :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
                 
                 AND
                 
                 EXTRACT(DOW FROM  citibike.starttime) >= 5
GROUP BY start_station_id, start_station_latitude, start_station_longitude) AS Subquery
                 '''


In [ ]:
dfAvgTripDur_WE_TS_MAX = pd.read_csv(StringIO.StringIO(queryCartoDB(qry14)), sep=',')
dfAvgTripDur_WE_TS_MAX.drop(['Unnamed: 1'], axis = 1, inplace=True)
dfAvgTripDur_WE_TS_MAX

#### Extra: Create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours
The number of trips per each pair of stations are output as attributes of these lines. 

In [ ]:


qry15 = '''
SELECT ST_Makeline(
            CDB_TransformToWebmercator(
                CDB_LatLng(start_station_latitude,
                       start_station_longitude 
                                                )),
          CDB_TransformToWebmercator(
                CDB_LatLng(end_station_latitude,
                       end_station_longitude 
                                                ))
                                                    ) 
                as the_geom_webmercator,
                                            MIN(cartodb_id) as cartodb_id,
                                            AVG(tripduration) as ta,
                                            COUNT(starttime) as tripcount
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                       start_station_longitude 
                                                )   :: geography,
                 CDB_LatLng(40.7577, -73.9857)  :: geography, 500) 
       AND tripduration < 7200
  
GROUP BY start_station_id, start_station_latitude, start_station_longitude,
             end_station_id, end_station_latitude, end_station_longitude 
'''                 


In [ ]:
dfLinePlots = pd.read_csv(StringIO.StringIO(queryCartoDB(qry15)), sep=',')
#dfAvgTripDur_WE_TS_MAX.drop(['Unnamed: 1'], axis = 1, inplace=True)
dfLinePlots.head()

## Lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours
![alt text](Trips.png "Trips started from stations within 500m of Times Square lasting less than 2 hours")

# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

## ASH - Score 100. Extra CRedit 0